In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

import warnings
warnings.filterwarnings("ignore")
import random

import hgd_dtw
import tensorflow as tf

## Heterogeneous Gower Distance
* IMPLEMENTATION OF GOWER DISTANCE CONSIDERING THE NATURE OF EACH VARIABLE, WITH CORRESPONDING NORMALIZATION AND RESCALING

## Features

In [ ]:
folders = ["s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10"
]

keys = ['Age', 'SAPSIIIScore', 'Gender']

binary = ['Gender']

continuous =  [variable for variable in keys if variable not in binary]

In [ ]:
for c in range(len(folders)):
    
    ########################
    ###### PARAMETERS ######
    ########################
    numberOfTimeStep = 1
    norm = "" #data is already normalized with minmax!

    # Load data
    X_train = pd.read_csv("../../../DATA/" + folders[c] + "/X_train_static" + norm + ".csv")
    X_train = X_train.to_numpy()
    X_train = X_train.reshape(X_train.shape[0], numberOfTimeStep, X_train.shape[1])


    print("X_noAMR_tr shape before reshape and val:", X_train.shape)
    
    # Reshape data by features --> PxTxF to FxTxP
    X_train = hgd_dtw.reshape_patients_by_features(X_train, keys, numberOfTimeStep)
    print("X_train shape after reshape:", X_train.shape)

    for nt in range(numberOfTimeStep):
        # Option 1: Generate a correlation matrix by matrix
        x = X_train[:, nt, :] 
        
        # Initialize distance matrix
        mat_by_time_step = np.zeros((x.shape[0], x.shape[0]))

        for i in range(x.shape[0]):
            for j in range(x.shape[0]):
                mat_by_time_step[i, j] = hgd_dtw.hgd_distance(x[i], x[j], keys[i], keys[j], binary, continuous)

        S = hgd_dtw.diagonal_to_zero(mat_by_time_step)

        pd.DataFrame(S, columns=keys, index=keys).to_csv("./estimatedGraphs/dtw-hgd/" + folders[c] + f"/adj_X_train_static{norm}" + ".csv", index=False)
